# Live Monte Carlo Stock Price Simulation

This notebook creates a live Monte Carlo simulation of stock prices that updates every second using Google Charts. The simulation will:

1. Generate random stock price movements based on geometric Brownian motion
2. Display multiple simulation paths
3. Update the chart in real-time
4. Serve the visualization through a local web server

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import json
import time
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
import socketserver
import os
import webbrowser
from IPython.display import display, HTML
print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
class MonteCarloStockSimulation:
    def __init__(self, initial_price=100, volatility=0.2, drift=0.05, num_simulations=5):
        """
        Initialize Monte Carlo stock simulation
        
        Parameters:
        - initial_price: Starting stock price
        - volatility: Annual volatility (standard deviation)
        - drift: Annual expected return (drift)
        - num_simulations: Number of simulation paths
        """
        self.initial_price = initial_price
        self.volatility = volatility
        self.drift = drift
        self.num_simulations = num_simulations
        self.current_prices = [initial_price] * num_simulations
        self.start_time = datetime.now()
        self.data_history = []
        
    def generate_next_prices(self):
        """Generate next set of prices using geometric Brownian motion"""
        dt = 1/252/24/3600  # Time step (1 second in trading years)
        
        # Generate random shocks
        random_shocks = np.random.normal(0, 1, self.num_simulations)
        
        # Calculate price changes using geometric Brownian motion
        # dS = S * (μ*dt + σ*sqrt(dt)*ε)
        price_changes = []
        for i, price in enumerate(self.current_prices):
            change = price * (self.drift * dt + self.volatility * np.sqrt(dt) * random_shocks[i])
            new_price = price + change
            # Ensure price doesn't go negative
            new_price = max(new_price, 0.01)
            price_changes.append(new_price)
        
        self.current_prices = price_changes
        return self.current_prices
    
    def get_current_data(self):
        """Get current simulation data formatted for Google Charts"""
        current_time = datetime.now()
        time_elapsed = (current_time - self.start_time).total_seconds()
        
        # Create data point
        data_point = {
            'time': current_time.strftime('%H:%M:%S'),
            'timestamp': time_elapsed,
            'prices': self.current_prices.copy()
        }
        
        self.data_history.append(data_point)
        
        # Keep only last 60 seconds of data
        if len(self.data_history) > 60:
            self.data_history = self.data_history[-60:]
        
        return data_point

# Initialize simulation
sim = MonteCarloStockSimulation(initial_price=100, volatility=0.3, drift=0.1, num_simulations=5)
print("Monte Carlo simulation initialized!")

Monte Carlo simulation initialized!


In [3]:
def create_html_template():
    """Create HTML template with Google Charts for live updating"""
    html_template = """
<!DOCTYPE html>
<html>
<head>
    <title>Live Monte Carlo Stock Simulation</title>
    <script type="text/javascript" src="https://www.gstatic.com/charts/loader.js"></script>
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 20px;
            background-color: #f5f5f5;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background-color: white;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .header {
            text-align: center;
            margin-bottom: 30px;
        }
        .stats {
            display: flex;
            justify-content: space-around;
            margin: 20px 0;
            padding: 15px;
            background-color: #f8f9fa;
            border-radius: 5px;
        }
        .stat {
            text-align: center;
        }
        .stat-value {
            font-size: 24px;
            font-weight: bold;
            color: #2e7d32;
        }
        .stat-label {
            font-size: 12px;
            color: #666;
        }
        #chart_div {
            width: 100%;
            height: 500px;
            margin: 20px 0;
        }
        .controls {
            text-align: center;
            margin: 20px 0;
        }
        button {
            padding: 10px 20px;
            margin: 5px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 14px;
        }
        .start-btn {
            background-color: #4CAF50;
            color: white;
        }
        .stop-btn {
            background-color: #f44336;
            color: white;
        }
        .reset-btn {
            background-color: #2196F3;
            color: white;
        }
        #status {
            text-align: center;
            margin: 10px 0;
            padding: 10px;
            border-radius: 5px;
        }
        .running {
            background-color: #dff0d8;
            color: #3c763d;
        }
        .stopped {
            background-color: #f2dede;
            color: #a94442;
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🎯 Live Monte Carlo Stock Price Simulation</h1>
            <p>Real-time simulation of stock price movements using geometric Brownian motion</p>
        </div>
        
        <div id="status" class="running">● RUNNING - Updating every second</div>
        
        <div class="controls">
            <button class="start-btn" onclick="startSimulation()">▶ Start</button>
            <button class="stop-btn" onclick="stopSimulation()">⏸ Pause</button>
            <button class="reset-btn" onclick="resetSimulation()">🔄 Reset</button>
        </div>
        
        <div class="stats">
            <div class="stat">
                <div class="stat-value" id="currentTime">--:--:--</div>
                <div class="stat-label">Current Time</div>
            </div>
            <div class="stat">
                <div class="stat-value" id="avgPrice">$0.00</div>
                <div class="stat-label">Average Price</div>
            </div>
            <div class="stat">
                <div class="stat-value" id="maxPrice">$0.00</div>
                <div class="stat-label">Highest Price</div>
            </div>
            <div class="stat">
                <div class="stat-value" id="minPrice">$0.00</div>
                <div class="stat-label">Lowest Price</div>
            </div>
        </div>
        
        <div id="chart_div"></div>
        
        <div style="text-align: center; margin-top: 20px; color: #666;">
            <p>Simulation Parameters: Initial Price = $100, Volatility = 30%, Annual Return = 10%</p>
        </div>
    </div>

    <script type="text/javascript">
        google.charts.load('current', {'packages':['line']});
        google.charts.setOnLoadCallback(drawChart);

        let chart;
        let data;
        let options;
        let isRunning = true;
        let simulationData = [];
        let startTime = new Date();

        function drawChart() {
            data = new google.visualization.DataTable();
            data.addColumn('number', 'Time (seconds)');
            data.addColumn('number', 'Path 1');
            data.addColumn('number', 'Path 2');
            data.addColumn('number', 'Path 3');
            data.addColumn('number', 'Path 4');
            data.addColumn('number', 'Path 5');

            options = {
                chart: {
                    title: 'Monte Carlo Stock Price Simulation Paths',
                    subtitle: 'Multiple simulation paths showing potential price movements'
                },
                width: '100%',
                height: 500,
                axes: {
                    x: {
                        0: {
                            label: 'Time (seconds since start)'
                        }
                    },
                    y: {
                        0: {
                            label: 'Stock Price ($)'
                        }
                    }
                },
                series: {
                    0: {color: '#1f77b4'},
                    1: {color: '#ff7f0e'},
                    2: {color: '#2ca02c'},
                    3: {color: '#d62728'},
                    4: {color: '#9467bd'}
                }
            };

            chart = new google.charts.Line(document.getElementById('chart_div'));
            chart.draw(data, google.charts.Line.convertOptions(options));
            
            // Start the simulation
            updateSimulation();
        }

        function updateSimulation() {
            if (!isRunning) return;
            
            fetch('/data')
                .then(response => response.json())
                .then(newData => {
                    // Add new data point
                    simulationData.push(newData);
                    
                    // Keep only last 60 seconds of data
                    if (simulationData.length > 60) {
                        simulationData = simulationData.slice(-60);
                    }
                    
                    // Update chart data
                    data = new google.visualization.DataTable();
                    data.addColumn('number', 'Time (seconds)');
                    data.addColumn('number', 'Path 1');
                    data.addColumn('number', 'Path 2');
                    data.addColumn('number', 'Path 3');
                    data.addColumn('number', 'Path 4');
                    data.addColumn('number', 'Path 5');
                    
                    // Add all data points
                    for (let i = 0; i < simulationData.length; i++) {
                        let point = simulationData[i];
                        data.addRow([
                            point.timestamp,
                            point.prices[0],
                            point.prices[1],
                            point.prices[2],
                            point.prices[3],
                            point.prices[4]
                        ]);
                    }
                    
                    chart.draw(data, google.charts.Line.convertOptions(options));
                    
                    // Update statistics
                    updateStats(newData);
                })
                .catch(error => {
                    console.error('Error fetching data:', error);
                });
            
            // Schedule next update
            setTimeout(updateSimulation, 1000);
        }

        function updateStats(newData) {
            // Update current time
            document.getElementById('currentTime').textContent = newData.time;
            
            // Calculate statistics
            let prices = newData.prices;
            let avgPrice = prices.reduce((a, b) => a + b, 0) / prices.length;
            let maxPrice = Math.max(...prices);
            let minPrice = Math.min(...prices);
            
            // Update display
            document.getElementById('avgPrice').textContent = '$' + avgPrice.toFixed(2);
            document.getElementById('maxPrice').textContent = '$' + maxPrice.toFixed(2);
            document.getElementById('minPrice').textContent = '$' + minPrice.toFixed(2);
        }

        function startSimulation() {
            isRunning = true;
            document.getElementById('status').className = 'running';
            document.getElementById('status').textContent = '● RUNNING - Updating every second';
            updateSimulation();
        }

        function stopSimulation() {
            isRunning = false;
            document.getElementById('status').className = 'stopped';
            document.getElementById('status').textContent = '⏸ PAUSED - Click start to resume';
        }

        function resetSimulation() {
            simulationData = [];
            startTime = new Date();
            fetch('/reset', {method: 'POST'});
            if (isRunning) {
                updateSimulation();
            }
        }
    </script>
</body>
</html>
    """
    return html_template

print("HTML template created!")

HTML template created!


In [4]:
class SimulationHTTPHandler(SimpleHTTPRequestHandler):
    """Custom HTTP handler for serving simulation data"""
    
    def __init__(self, *args, simulation=None, **kwargs):
        self.simulation = simulation
        super().__init__(*args, **kwargs)
    
    def do_GET(self):
        if self.path == '/':
            # Serve the main HTML page
            self.send_response(200)
            self.send_header('Content-type', 'text/html')
            self.end_headers()
            html_content = create_html_template()
            self.wfile.write(html_content.encode())
        
        elif self.path == '/data':
            # Serve simulation data as JSON
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.send_header('Access-Control-Allow-Origin', '*')
            self.end_headers()
            
            # Generate next prices and get current data
            self.simulation.generate_next_prices()
            current_data = self.simulation.get_current_data()
            
            json_data = json.dumps(current_data)
            self.wfile.write(json_data.encode())
        
        else:
            # Serve other files normally
            super().do_GET()
    
    def do_POST(self):
        if self.path == '/reset':
            # Reset simulation
            self.simulation.__init__(
                initial_price=100, 
                volatility=0.3, 
                drift=0.1, 
                num_simulations=5
            )
            self.send_response(200)
            self.send_header('Content-type', 'application/json')
            self.end_headers()
            self.wfile.write(b'{"status": "reset"}')
        else:
            self.send_response(404)
            self.end_headers()
    
    def log_message(self, format, *args):
        # Suppress server logs to keep notebook output clean
        pass

def create_server(simulation, port=8080):
    """Create and configure the HTTP server"""
    def handler(*args, **kwargs):
        return SimulationHTTPHandler(*args, simulation=simulation, **kwargs)
    
    server = HTTPServer(('localhost', port), handler)
    return server

print("HTTP server handler created!")

HTTP server handler created!


In [5]:
def start_live_simulation(port=8080, auto_open=True):
    """
    Start the live Monte Carlo simulation server
    
    Parameters:
    - port: Port number for the web server (default: 8080)
    - auto_open: Whether to automatically open the browser (default: True)
    """
    
    try:
        # Create server
        server = create_server(sim, port)
        
        print(f"🚀 Starting Monte Carlo simulation server on port {port}...")
        print(f"📊 Access the live chart at: http://localhost:{port}")
        print(f"⏱️  Chart updates every second with new data points")
        print(f"📈 Showing {sim.num_simulations} simulation paths")
        print("\n" + "="*60)
        print("SIMULATION PARAMETERS:")
        print(f"📍 Initial Price: ${sim.initial_price}")
        print(f"📊 Volatility: {sim.volatility*100:.1f}% (annual)")
        print(f"📈 Expected Return: {sim.drift*100:.1f}% (annual)")
        print(f"🎯 Number of Paths: {sim.num_simulations}")
        print("="*60)
        print("\n💡 The chart will show multiple price paths evolving in real-time!")
        print("🎮 Use the Start/Pause/Reset buttons on the webpage to control the simulation")
        print("\n⚠️  To stop the server, interrupt the kernel or press Ctrl+C")
        
        # Open browser automatically
        if auto_open:
            import threading
            def open_browser():
                time.sleep(1)  # Wait a moment for server to start
                webbrowser.open(f'http://localhost:{port}')
            
            browser_thread = threading.Thread(target=open_browser)
            browser_thread.daemon = True
            browser_thread.start()
        
        # Start serving
        print(f"\n🌐 Server starting... Opening browser in 1 second...")
        server.serve_forever()
        
    except KeyboardInterrupt:
        print("\n🛑 Simulation stopped by user")
        server.shutdown()
    except Exception as e:
        print(f"❌ Error starting server: {e}")
        print("💡 Try using a different port number if port 8080 is busy")

print("Main simulation function ready!")

Main simulation function ready!


## 🚀 Ready to Launch!

**To start the live simulation, run the cell below.** 

The simulation will:
- ✅ Start a local web server on port 8080
- ✅ Automatically open your browser
- ✅ Display 5 different Monte Carlo simulation paths
- ✅ Update the chart every second with new price movements
- ✅ Show real-time statistics (average, min, max prices)

**Controls available on the webpage:**
- ▶️ **Start/Pause**: Control the simulation
- 🔄 **Reset**: Restart with initial conditions
- 📊 **Live Stats**: See current price statistics

**Technical Details:**
- Uses **Geometric Brownian Motion** for realistic stock price modeling
- **Google Charts** for smooth, interactive visualization
- **Real-time updates** every second
- **60-second rolling window** to keep chart responsive

In [6]:
# 🎯 START THE LIVE SIMULATION!
# Run this cell to launch the live Monte Carlo simulation
# The browser will open automatically showing the real-time chart

print("🎲 Launching Monte Carlo Live Simulation...")
print("⏰ Starting in 3 seconds...")

# Small delay to show the message
time.sleep(3)

# Start the simulation!
start_live_simulation(port=8080, auto_open=True)

🎲 Launching Monte Carlo Live Simulation...
⏰ Starting in 3 seconds...
🚀 Starting Monte Carlo simulation server on port 8080...
📊 Access the live chart at: http://localhost:8080
⏱️  Chart updates every second with new data points
📈 Showing 5 simulation paths

SIMULATION PARAMETERS:
📍 Initial Price: $100
📊 Volatility: 30.0% (annual)
📈 Expected Return: 10.0% (annual)
🎯 Number of Paths: 5

💡 The chart will show multiple price paths evolving in real-time!
🎮 Use the Start/Pause/Reset buttons on the webpage to control the simulation

⚠️  To stop the server, interrupt the kernel or press Ctrl+C

🌐 Server starting... Opening browser in 1 second...

🛑 Simulation stopped by user
